# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter der Bischöfe einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: `bischof.tbl_bischoefe_gatz` und `bischof.tbl_bischoefe_gatzaemter`. Alternativ werden die Daten in CSV-Dateien geschrieben und von dort aus hier eingelesen.

[Amt](#Amt)  
[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt)





Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path = "../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [12]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-05-18"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-05-18"

In [13]:
item_type_id = 4

4

In [14]:
ENV["COLUMNS"] = 120

120

*2022-01-13* Für die offizielle Liste der Ämter gibt es inzwischen eine Liste: `domherr.tbl_aemterliste_normiert`. Die Ämter der Bischöfe sind neu zuzuordnen.

## Amtsperiode

In [10]:
# Quelle Datenbank
sql = "SELECT * FROM bischof.tbl_bischoefe_gatzaemter"
df_pr = Wds.sql_df(sql);

In [10]:
using CSV, DataFrames

In [54]:
# Quelle CSV
filename = joinpath(data_path, "tbl_bischoefe_gatzaemter.csv");
df_pr = CSV.read(filename, DataFrame, stringtype = String);

In [55]:
size(df_pr)

(7339, 15)

In [56]:
names(df_pr)

15-element Vector{String}:
 "Bistum"
 "Amtsart"
 "Amtsbeginn"
 "Amtsende"
 "ID_Amt"
 "ID_Bischof"
 "Institution"
 "Amt_weltlich"
 "Herrschaftsgebiet"
 "Ort"
 "Amt_Kommentar"
 "Sort"
 "Titularbistum"
 "Profession"
 "id_gsn"

nicht befüllt:
- Herrschafsgebiet
- Institution
- Amt_weltlich
- Profession
- Ort

### Bischöfe zuordnen

In [57]:
transform!(df_pr, :ID_Bischof => ByRow(string) => :id_in_source);

In [58]:
bishop_item_type = 4;
sql = "SELECT id AS person_id, id_in_source FROM item WHERE item_type_id = $(bishop_item_type)";
df_idx = Wds.sql_df(sql);

In [59]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

In [60]:
size(df_prp)

(7339, 17)

Amtsangaben ohne Bezug zu einem Bischof

In [61]:
sort_ID_Bischof(df) = sort(df, :ID_Bischof)
filter(:person_id => ismissing, df_prp) |> sort_ID_Bischof

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing
1,Brixen,Weihbischof,1379,1387,22075,21066,missing,missing,missing
2,Chur,Weihbischof,um 1289,1294,22087,21078,missing,missing,missing
3,Eichstätt,Weihbischof,1340,missing,22102,21093,missing,missing,missing
4,Halberstadt,Weihbischof,1420,1436,22144,21135,missing,missing,missing
5,Köln,Weihbischof,1228,missing,22173,21164,missing,missing,missing
6,Köln,Weihbischof,1352,1364,22184,21175,missing,missing,missing
7,Konstanz,Weihbischof,1289,1294 (1299 ?),22198,21189,missing,missing,missing
8,Leitomischl,Weihbischof,1361,missing,22238,21229,missing,missing,missing
9,Lüttich,Weihbischof,1233,missing,22243,21234,missing,missing,missing


19 Amtsperioden können keinem Bischof zugeordnet werden?!

In [62]:
df_prp = dropmissing(df_prp, :person_id);

In [63]:
size(df_prp)

(7320, 17)

Lösche Einträge ohne Amtsbezeichnung

In [64]:
df_prp = dropmissing(df_prp, :Amtsart);

In [65]:
size(df_prp)

(7320, 17)

### Rollen/Ämter zuordnen

In [66]:
sql = "SELECT name, id FROM role"
df_role = Wds.sql_df(sql);

In [67]:
lookup_role = Dict(df_role.name .=> df_role.id);

In [68]:
get_role_id(name) = get(lookup_role, name, missing)

get_role_id (generic function with 1 method)

In [69]:
get_role_id("Vikar")

249238

Die Einträge in Tabelle `role` beginnen alle mit einem Großbuchstaben

In [70]:
starts_lowercase(s) = islowercase(s[1]);
df_prp_sl = filter(:Amtsart => starts_lowercase, df_prp);

In [71]:
select(df_prp_sl, [:person_id, :id_in_source, :Amtsart, :Bistum])

,person_id,id_in_source,Amtsart,Bistum
,Int32,String,String,String?
1,266924,10429,nominierter Bischof,Lüttich
2,267203,4254,designierter Bischof,Konstanz
3,269987,21771,evangelischer Administrator,Bremen-Hamburg
4,269988,21772,evangelischer Administrator,Bremen-Hamburg
5,269989,21773,evangelischer Administrator,Bremen-Hamburg


In [72]:
transform!(df_prp, :Amtsart => ByRow(uppercasefirst) => :Amtsart);

SQL berücksichtigt keine Unterschiede in Groß- und Kleinschreibung

In [73]:
df_info = subset(df_prp, :ID_Bischof => ByRow(isequal(10127)))

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing
1,Münster,Fürstbischof,1719,1761,5609,10127,missing,missing,missing
2,Regensburg,Fürstbischof,1717,1719,4222,10127,missing,missing,missing
3,Paderborn,Fürstbischof,1719,1761,4224,10127,missing,missing,missing
4,Köln,Koadjutor des Erzbischofs,1722,1723,4225,10127,missing,missing,missing
5,Köln,Kurfürst-Erzbischof,1723,1761,4226,10127,missing,missing,missing
6,Hildesheim,Fürstbischof,1724,1761,4227,10127,missing,missing,missing
7,Osnabrück,Fürstbischof,1728,1761,4228,10127,missing,missing,missing
8,missing,Fürstpropst,1723,1761,21892,10127,missing,missing,missing
9,Regensburg,Koadjutor,1715,1716,23084,10127,missing,missing,missing


Für Bischof 3807 sind Amtsart und Bistum vertauscht. *2022-05-19* in der Quelle korrigiert.

In [74]:
subset(df_pr, :ID_Bischof => ByRow(isequal(10127)))

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing
1,Münster,Fürstbischof,1719,1761,5609,10127,missing,missing,missing
2,Regensburg,Fürstbischof,1717,1719,4222,10127,missing,missing,missing
3,Paderborn,Fürstbischof,1719,1761,4224,10127,missing,missing,missing
4,Köln,Koadjutor des Erzbischofs,1722,1723,4225,10127,missing,missing,missing
5,Köln,Kurfürst-Erzbischof,1723,1761,4226,10127,missing,missing,missing
6,Hildesheim,Fürstbischof,1724,1761,4227,10127,missing,missing,missing
7,Osnabrück,Fürstbischof,1728,1761,4228,10127,missing,missing,missing
8,missing,Fürstpropst,1723,1761,21892,10127,missing,missing,missing
9,Regensburg,Koadjutor,1715,1716,23084,10127,missing,missing,missing


Ämter der Ämterliste zuordnen

In [75]:
transform!(df_prp, :Amtsart => ByRow(get_role_id) => :role_id);

In [76]:
df_role_mg = filter(:role_id => ismissing, df_prp);

In [77]:
length(unique(df_role_mg.Amtsart))

98

In [78]:
unique(df_role_mg[!, :Amtsart])[1:7]

7-element Vector{String}:
 "Fürstpropst"
 "Direktor des Konsistoriums"
 "Kurfürst-Erzbischof"
 "Apostolischer Administrator"
 "Fürstbischof"
 "Koadjutor des Bischofs"
 "Koadjutor des Kurfürst-Erzbischofs"

In diesen Fällen wird die Amtsart lediglich eingetragen in `person_role.role_name`

### Bistümer zuordnen

In [79]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese";
df_dioc = Wds.sql_df(sql);

In [80]:
df_prp = leftjoin(df_prp, df_dioc, on = :Bistum => :diocese_name, matchmissing = :notequal);

In [81]:
df_dioc_mg = filter(:diocese_id => ismissing, df_prp);

In [82]:
size(df_dioc_mg)

(447, 19)

Numerische Daten eintragen

In [83]:
num_date_begin(s_date) = ismissing(s_date) ? s_date : Wds.parsemaybe(String(s_date), :lower)

num_date_begin (generic function with 1 method)

In [84]:
num_date_end(s_date) = ismissing(s_date) ? s_date : Wds.parsemaybe(String(s_date), :upper)

num_date_end (generic function with 1 method)

In [86]:
df_prp[201:205, :]

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing
1,Trier,Kurfürst-Erzbischof,1756,1768,5674,10860,missing,missing,missing
2,Worms,Fürstbischof,1763,1768,5675,10860,missing,missing,missing
3,Wien,Administrator,1476,1482,5676,5302,missing,missing,missing
4,Salzburg,Administrator,1481,1487,5677,5302,missing,missing,missing
5,Magdeburg,Erzbischof,1125/1126,missing,5678,5080,missing,missing,missing


In [87]:
transform!(df_prp, :Amtsbeginn => ByRow(num_date_begin) => :num_date_begin);

┌ Warning: Could not parse 
│   s = fränkische Zeit ?
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1118


In [88]:
transform!(df_prp, :Amtsende => ByRow(num_date_end) => :num_date_end);

In [89]:
df_prp[561:565, [:ID_Bischof, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

,ID_Bischof,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int64,String,String?,Int64?,String?,Int64?
1,10475,Kurfürst-Erzbischof,1784,1784,1801,1801
2,10476,Fürstbischof,1650,1650,1688,1688
3,3302,Elekt,1387,1387,1389,1389
4,3307,Weihbischof,1279,1279,1280,1280
5,3307,Weihbischof,1279,1279,1285,1285


Schlüssel für eine zeitliche Sortierung eintragen

In [90]:
transform!(df_prp, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = (1168) 1180
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (1014 ?) 1015
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (952)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (972)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (984)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (991)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (996)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 

In [91]:
transform!(df_prp, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: could not parse 
│   s = (1014 ?) 1015
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (952)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (972)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (984)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (991)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (996)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   s = (1072)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1527
┌ Warning: could not parse 
│   

In [92]:
replace_if_missing(a, b) = ismissing(a) ? b : a

replace_if_missing (generic function with 1 method)

In [93]:
transform!(df_prp, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

In [94]:
size(df_prp, 1), count(ismissing, df_prp.date_sort_key)

(7320, 0)

Bestehende Einträge löschen

In [95]:
table_name = "person_role";
item_type_id = 4;
sql = "DELETE FROM $(table_name) WHERE person_id IN (SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN (SELECT id FROM item WHERE item_type_id = 4)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer, auch für die identifizierten, den Namen des Bistums in die Tabelle.  
Schreibe für alle Ämter, auch für die identifizierten, den Namen des Amts in die Tabelle.

In [96]:
columns = [
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [97]:
table_dst = "person_role"
Wds.filltable!(table_dst, select(df_prp, columns))

┌ Info: Rows inserted: 7320
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


7320

Titularbistum eintragen

Identifikation über `ID_Bischof`, `Amtsart`, `Bistum`

Es gibt nur einen Eintrag, der nicht leer ist?!
``` sql
SELECT ID_Bischof, Amtsart, Bistum, COUNT(*) AS n
FROM bischof.tbl_bischoefe_gatzaemter
WHERE Titularbistum IS NOT NULL AND Titularbistum <> ''
GROUP BY ID_Bischof, Amtsart, Bistum
ORDER BY n DESC;
```

Es gibt nur einen Eintrag für `ID_Bischof` = 21596. Dieser Bischof ist aber nicht in der Bischofsliste.

In [98]:
filter(:ID_Bischof => isequal(21596), select(df_prp, :ID_Bischof, :Bistum, :Amtsart, :Titularbistum))

,ID_Bischof,Bistum,Amtsart,Titularbistum
,Int64,String?,String,String?


## Zeitspanne der Person

In [99]:
sql = "SELECT id, num_date_birth, num_date_death
FROM person WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [100]:
size(df_date)

(5171, 3)

Zeitspanne der Ämter

In [101]:
df_pr_begin_value = dropmissing(df_prp, :num_date_begin);
df_pr_begin = groupby(select(df_pr_begin_value, :person_id, :num_date_begin), :person_id);

In [102]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [103]:
size(df_pr_date_min)

(5025, 2)

In [104]:
names(df_pr_date_min)

2-element Vector{String}:
 "person_id"
 "date_min"

In [105]:
df_pr_end_value = dropmissing(df_prp, :num_date_end);
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id);

In [106]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [107]:
size(df_pr_date_max)

(4539, 2)

In [108]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [109]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

In [110]:
size(df_date)

(5171, 5)

In [111]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [112]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [113]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [114]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [115]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [116]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [117]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [118]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein 

In [119]:
sql = "CREATE TABLE date_update (id INT, date_min INT, date_max INT) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "CREATE TABLE date_update (id INT, date_min INT, date_max INT) \nENGINE=InnoDB DEFAULT CHARSET=utf8;", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [121]:
columns = [
    :id,
    :date_min,
    :date_max,
]

3-element Vector{Symbol}:
 :id
 :date_min
 :date_max

In [123]:
size(df_date)

(5038, 5)

In [ ]:
table_name = "date_update"
Wds.filltable!(table_name, select(df_date, columns))

*2022-01-14* Fehler `UndefRefError: access to undefined reference` mit `Wds.filltable!`  
Füge die Zeilen einzeln ein.

In [127]:
table_name = "date_update";
# Es gibt keine fehlenden Werte
delta_msg = 400
for (i, row) in enumerate(eachrow(df_date))
    (id, date_min, date_max) = row[[:id, :date_min, :date_max]]
    sql = "INSERT INTO $(table_name) VALUES ($(id), $(date_min), $(date_max))";
    DBInterface.execute(Wds.dbwiag, sql)
    if i % delta_msg == 0
       @info i
    end
end

┌ Info: 400
└ @ Main In[127]:9
┌ Info: 800
└ @ Main In[127]:9
┌ Info: 1200
└ @ Main In[127]:9
┌ Info: 1600
└ @ Main In[127]:9
┌ Info: 2000
└ @ Main In[127]:9
┌ Info: 2400
└ @ Main In[127]:9
┌ Info: 2800
└ @ Main In[127]:9
┌ Info: 3200
└ @ Main In[127]:9
┌ Info: 3600
└ @ Main In[127]:9
┌ Info: 4000
└ @ Main In[127]:9
┌ Info: 4400
└ @ Main In[127]:9
┌ Info: 4800
└ @ Main In[127]:9


*2022-01-14* Schleife läuft fehlerfrei durch?!

In [128]:
sql = "UPDATE person AS p, 
(SELECT id, date_min, date_max FROM date_update) AS du
SET p.date_min = du.date_min, p.date_max = du.date_max
WHERE p.id = du.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "UPDATE person AS p, \n(SELECT id, date_min, date_max FROM date_update) AS du\nSET p.date_min = du.date_min, p.date_max = du.date_max\nWHERE p.id = du.id", 0, -1, 5038, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-14* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12